<a href="https://colab.research.google.com/github/resonansss/Domashki_sem3/blob/hw02_nn/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22hw02_nn_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [60]:
!pip install torchmetrics
import random
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.utils.data as data_utils
import torch.nn as nn
import torch.optim as optim
from torchmetrics.regression import MeanSquaredError
from torch.utils.data import Dataset, DataLoader, TensorDataset
import tqdm
import torch.nn.functional as F

In [61]:
# %conda install wget

In [62]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [63]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2024-01-08 15:18:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data.txt.zip’

data.txt.zip            [             <=>    ] 201.24M  74.5MB/s    in 2.7s    

2024-01-08 15:18:22 (74.5 MB/s) - ‘data.txt.zip’ saved [211011981]



In [64]:
df = pd.read_csv("data.txt.zip", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [65]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [66]:
# YOUR CODE HERE (－.－)...zzzZZZzzzZZZ

model = Ridge(alpha=0.05)
model.fit(X_train, y_train)

Ridge(alpha=0.05)

In [67]:
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)
print(f'качество рмсе: {rmse}')

качество рмсе: 90.44315668595496


(это лучше ^^^)

In [68]:
constant_pred = [y_train.mean()] * len(y_test)
rmse_constant = mean_squared_error(y_test, constant_pred)
print(f'качество рмсе констант: {rmse_constant}')

качество рмсе констант: 117.77597281228708


(это хуже ^^^)

## Задание 1. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из ячейки ниже.



In [69]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [70]:
scal_1 = StandardScaler()
scal_2 = MinMaxScaler()

X_train_1 = scal_1.fit_transform(X_train)
X_train_2 = scal_2.fit_transform(X_train)
X_test_1 = scal_1.transform(X_test)
X_test_2 = scal_2.transform(X_test)

Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться следующими сигнатурами функций. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [71]:
INPUT_SIZE = 90
HIDDEN_SIZE1 = 42
HIDDEN_SIZE2 = 42
OUTPUT_SIZE = 1
LEARNING_RATE = 0.005
EPOCHS = 20
BATCH_SIZE = 128


class ClassClass(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def create_data_loader(X_train, y_train, X_test, y_test):
    train_tensor = torch.utils.data.TensorDataset(torch.tensor(X_train.astype(np.float32)), torch.tensor(y_train))
    train_loader = torch.utils.data.DataLoader(dataset=train_tensor, batch_size=BATCH_SIZE, shuffle=True)
    test_tensor = torch.utils.data.TensorDataset(torch.tensor(X_test.astype(np.float32)), torch.tensor(y_test))
    test_loader = torch.utils.data.DataLoader(dataset=test_tensor, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader


class TorchModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=INPUT_SIZE, out_features=HIDDEN_SIZE1)
        self.fc2 = nn.Linear(in_features=HIDDEN_SIZE1, out_features=HIDDEN_SIZE2)
        self.out = nn.Linear(in_features=HIDDEN_SIZE2, out_features=OUTPUT_SIZE)
    def forward(self, x):
        x = nn.LeakyReLU()(self.fc1(x))
        x = nn.LeakyReLU()(self.fc2(x))
        x = self.out(x)
        return x

model = TorchModel()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()
train_loader, test_loader = create_data_loader(X_train_1, y_train, X_test_1, y_test)


In [72]:
def train_train(model, optimizer, criterion, train_loader, test_loader):

    for epoch in range(EPOCHS):
        model.train()
        for x_train, y_train in train_loader:
            y_pred = model(x_train).squeeze()
            loss = torch.sqrt(criterion(y_pred, y_train.float()))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        val_loss = []
        if epoch % 2 == 0:
            with torch.no_grad():
                for x_test, y_test in test_loader:
                    y_pred = model(x_test).squeeze()
                    loss = torch.sqrt(criterion(y_pred, y_test))
                    val_loss.append(loss.numpy())
            print(f'эпоха: {epoch}, лосс: {np.mean(val_loss)}')

train_train(model, optimizer, criterion, train_loader, test_loader)

эпоха: 0, лосс: 296.0013427734375
эпоха: 2, лосс: 261.4915466308594
эпоха: 4, лосс: 236.84629821777344
эпоха: 6, лосс: 15.031566619873047
эпоха: 8, лосс: 13.096823692321777
эпоха: 10, лосс: 13.478631973266602
эпоха: 12, лосс: 10.223864555358887
эпоха: 14, лосс: 14.155427932739258
эпоха: 16, лосс: 11.17469310760498
эпоха: 18, лосс: 12.882372856140137


кошмар.

(стандарт скалер) ^^^

In [73]:
train_loader, test_loader = create_data_loader(X_train_2, y_train, X_test_2, y_test)
train_train(model, optimizer, criterion, train_loader, test_loader)

эпоха: 0, лосс: 10.318625450134277
эпоха: 2, лосс: 11.260324478149414
эпоха: 4, лосс: 12.26105785369873
эпоха: 6, лосс: 9.606587409973145
эпоха: 8, лосс: 12.036405563354492
эпоха: 10, лосс: 9.918767929077148
эпоха: 12, лосс: 12.070539474487305
эпоха: 14, лосс: 9.891263008117676
эпоха: 16, лосс: 9.076273918151855
эпоха: 18, лосс: 11.342752456665039


(мин макс скалер) ^^^

тут хоть лосс не прыгает туда-сюда как сумасшедший.........

посмотрю на другую модель \\/

In [74]:
def model_model():
    model = nn.Sequential(
        nn.Linear(in_features=INPUT_SIZE, out_features=HIDDEN_SIZE1),
        nn.ReLU(),
        nn.Linear(in_features=HIDDEN_SIZE1, out_features=HIDDEN_SIZE2),
        nn.ReLU(),
        nn.Linear(in_features=HIDDEN_SIZE2, out_features=OUTPUT_SIZE),
    )

    return model

In [75]:
model = model_model()

In [76]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()
train_loader, test_loader = create_data_loader(X_train_1, y_train, X_test_1, y_test) #стандарт скалер
train_train(model, optimizer, criterion, train_loader, test_loader)

эпоха: 0, лосс: 188.1275177001953
эпоха: 2, лосс: 271.5869445800781
эпоха: 4, лосс: 197.4871368408203
эпоха: 6, лосс: 14.951356887817383
эпоха: 8, лосс: 12.971046447753906
эпоха: 10, лосс: 12.00437068939209
эпоха: 12, лосс: 12.508916854858398
эпоха: 14, лосс: 11.707050323486328
эпоха: 16, лосс: 11.49094009399414
эпоха: 18, лосс: 12.226639747619629


://////////

:\\\\\\\\\\\\\\\\\

:|||||||||||||

In [77]:
train_loader, test_loader = create_data_loader(X_train_2, y_train, X_test_2, y_test) #мин макс скалер
train_train(model, optimizer, criterion, train_loader, test_loader)

эпоха: 0, лосс: 9.808815956115723
эпоха: 2, лосс: 13.227006912231445
эпоха: 4, лосс: 9.370877265930176
эпоха: 6, лосс: 14.116782188415527
эпоха: 8, лосс: 9.165492057800293
эпоха: 10, лосс: 11.658432960510254
эпоха: 12, лосс: 10.628098487854004
эпоха: 14, лосс: 12.030010223388672
эпоха: 16, лосс: 12.709311485290527
эпоха: 18, лосс: 13.057463645935059


минмакс скалер работает (чуть) адекватнее, голосуйте за минмакс скалер

минмакс скалер -- наш выбор!

## Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

In [78]:
# YOUR CODE HERE (－.－)...zzzZZZzzzZZZ

отчет

1. это было довольно сложно, и я брала мало эпох, скорее всего нужно было брать хотя бы 50-60, больше -- лучше (главное не переучиться и понять, где нужно вовремя остановиться)

2. изначально нужно скалировать минмакс скалером, от него лосс сразу идет стабильный

3. модели: 3 слоя, 2 функции активации (в первом случае leakeyrelu, во втором -- relu), оптимайзер везде sgd. можно было попробовать другие, например, адам, но я не пробовала.

4. ниже будет нестеров=true с минмакс скалером, который покажет лучший результат: я попыталась потестить learning rate и добилась лосс в 8.490 (для минмакс скалер) и 8.167 (для стандарт скалер)



In [91]:
model = model_model()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True)
train_loader, test_loader = create_data_loader(X_train_2, y_train, X_test_2, y_test)

In [92]:
train_train(model, optimizer, criterion, train_loader, test_loader)

эпоха: 0, лосс: 58.21388626098633
эпоха: 2, лосс: 333.421630859375
эпоха: 4, лосс: 10.41161823272705
эпоха: 6, лосс: 13.05688762664795
эпоха: 8, лосс: 9.90872573852539
эпоха: 10, лосс: 9.950630187988281
эпоха: 12, лосс: 9.908416748046875
эпоха: 14, лосс: 9.97547435760498
эпоха: 16, лосс: 9.975668907165527
эпоха: 18, лосс: 9.975126266479492


In [106]:
model = model_model()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)
train_loader, test_loader = create_data_loader(X_train_2, y_train, X_test_2, y_test)
train_train(model, optimizer, criterion, train_loader, test_loader)

эпоха: 0, лосс: 100.80119323730469
эпоха: 2, лосс: 37.71623992919922
эпоха: 4, лосс: 226.1429901123047
эпоха: 6, лосс: 9.116411209106445
эпоха: 8, лосс: 8.717957496643066
эпоха: 10, лосс: 8.927793502807617
эпоха: 12, лосс: 8.9777250289917
эпоха: 14, лосс: 9.374176979064941
эпоха: 16, лосс: 8.868877410888672
эпоха: 18, лосс: 8.490281105041504


In [107]:
model = model_model()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)
train_loader, test_loader = create_data_loader(X_train_1, y_train, X_test_1, y_test)
train_train(model, optimizer, criterion, train_loader, test_loader)

эпоха: 0, лосс: 94.03248596191406
эпоха: 2, лосс: 65.7542953491211
эпоха: 4, лосс: 57.924766540527344
эпоха: 6, лосс: 72.32614135742188
эпоха: 8, лосс: 57.39958953857422
эпоха: 10, лосс: 68.92186737060547
эпоха: 12, лосс: 8.522953033447266
эпоха: 14, лосс: 8.339288711547852
эпоха: 16, лосс: 8.167990684509277
эпоха: 18, лосс: 8.213752746582031


![image.png](https://gifdb.com/images/high/oooo-498-x-278-gif-8i95ffhcb6glb7jd.gif)